In [16]:
# -*- coding: utf-8 -*- 

import pandas as pd
import numpy as np
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from xgboost.sklearn import XGBClassifier
import string



import sys


In [17]:

# se podria hacer el mismo codigo pero en 4 particiones del tiempo

train=pd.read_csv("D:\\data science game\\train.csv")
test=pd.read_csv("D:\\data science game\\test.csv")





In [18]:
#canciones=train.append(test,ignore_index=True)
#canciones=canciones.loc[canciones.loc[:,"listen_type"]==1,]
#canciones=canciones[["user_id","media_id","ts_listen","is_listened"]]
#canciones["ts_listen2"]=pd.to_datetime(canciones["ts_listen"], unit='s')

#canciones.sort_values(["user_id","media_id","ts_listen2"],ascending=[True,True,True],inplace=True)
#canciones.reset_index(drop=True,inplace=True)

#canciones["indicator"]=1

#canciones["anterior_listened"]=canciones.groupby(['user_id',"media_id"])['is_listened'].shift(1)

#canciones["freq_cancion_recomendada"]=canciones.groupby(['user_id',"media_id"])["indicator"].cumsum()

#canciones['user_id-media_id_prob_target']=canciones.groupby(['user_id',"media_id"])["anterior_listened"].expanding().mean().reset_index(drop=True)

#canciones.loc[canciones["freq_cancion_recomendada"] <= 7,'user_id-media_id_prob_target']=np.nan 

In [19]:
#del canciones

In [20]:
json_music=pd.read_csv("D:\\data science game\\cancionesyalbumnes.csv",encoding="utf-8" )


In [21]:
len(json_music)

453310

In [22]:
new_cols = ['Remastered',"Tribute to","Version","Edition","Deluxe","Special"] #edition,delux

for col in new_cols:
    json_music[col+"_song"] = json_music['sng_title'].apply(lambda x: 1 if col in x else 0)
    
new_cols = ['remix',"Remix"]

for col in new_cols:
    json_music["Remix_song"] = json_music['sng_title'].apply(lambda x: 1 if col in x else 0)

new_cols = ['feat', "Feat"] #edition,delux

for col in new_cols:
    json_music["Feat_song"] = json_music['sng_title'].apply(lambda x: 1 if col in x else 0)

    
new_cols = ['- Live', '( Live', '(Live']

for col in new_cols:
    json_music["Live_song"] = json_music['sng_title'].apply(lambda x: 1 if col in x else 0)
    
    
string.punctuation.__add__('!!')
string.punctuation.__add__('(')
string.punctuation.__add__(')')

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

json_music['sng_title_cleaned'] = json_music['sng_title'].apply(lambda x: x.translate(remove_punct_map))
json_music['letters_count_song'] = json_music['sng_title_cleaned'].apply(lambda x: len(x.strip()))
json_music['words_count_song'] = json_music['sng_title_cleaned'].apply(lambda x: 0 if len(x.strip()) == 0 else len(x.split(' ')))
json_music["letters_count/words_count"]=json_music['letters_count_song']*1.0/json_music['words_count_song']
    
json_music["quantile_words_count_song"]=pd.cut(json_music["words_count_song"],4)
json_music["quantile_letters_count_song"]=pd.cut(json_music["letters_count_song"],4)
json_music["quantile_letters_count/words_count"]=pd.cut(json_music["letters_count/words_count"],4)



    

In [23]:
def compare_target(unidades,variable,target,function,param_train,param_test,test_ofic):
    temp_train=param_train[param_train[target[0]]==1]
    var_comp = unidades+variable
    listf=[function,"std","size"]
    ind=len(var_comp)-1
    
    info = temp_train[var_comp].groupby(unidades).agg(listf).reset_index()
    info.columns = info.columns.get_level_values(0)
    info.columns.values[ind]=variable[0]+"_target"
    info.columns.values[ind+1]=variable[0]+"std_target"
    info.columns.values[ind+2]="count"
    info=info.loc[info["count"]>=20,]
    info.drop('count', axis=1, inplace=True)

    param_train=param_train.merge(info,on=unidades,how="left")
    param_test=param_test.merge(info,on=unidades,how="left")
    test_ofic=test_ofic.merge(info,on=unidades,how="left")

    
    return param_train, param_test , test_ofic

def cat_target(var_cat,target,param_train,param_test,estimate_train=1):
    gr=var_cat+target
    temp=param_train[gr].groupby(var_cat).agg(["mean","size"]).reset_index()
    temp.columns = temp.columns.get_level_values(0)
    temp.columns.values[len(var_cat)]='-'.join(var_cat)+"_prob_target"
    temp.columns.values[len(var_cat)+1]="count"
    temp=temp.loc[temp.loc[:,"count"]>=20,]
    temp.drop('count', axis=1, inplace=True)

    param_test=param_test.merge(temp,on=var_cat,how="left")
    if estimate_train==1:
        param_train=param_train.merge(temp,on=var_cat,how="left")
        return param_train,param_test
    else:
        return param_test
    
    
merge 


def estimate_cross(train,test,var_cat,target):
    n=len(train)
    a,b=cat_target(var_cat,target,param_train=train.loc[0:(n/4),],param_test=train.loc[(n/4)+1:(n/2),])
    c=cat_target(var_cat,target,param_train=train.loc[0:(n/2),],param_test=train.loc[(n/2)+1:(3*n/4),],estimate_train=0)
    d=cat_target(var_cat,target,param_train=train.loc[0:(3*n/4),],param_test=train.loc[(3*n/4)+1:n,],estimate_train=0)
    test=cat_target(var_cat,target,param_train=train,param_test=test,estimate_train=0)
    
    train_new=pd.concat([a,b,c,d],ignore_index=True)
    test_new=test

    return train_new,test_new


def classify_daytime(hour):
    if hour <= 6:
        return 'night'
    elif hour <= 12:
        return 'morning'
    elif hour <= 18:
        return 'afternoon'
    elif hour <= 23:
        return 'evening'



In [24]:

train2=train[train["listen_type"]==1].copy()
test2=test.copy()
train2["provenencia"]="train"
test2["provenencia"]="test"

temp = train2[["user_id","media_id","ts_listen","provenencia","media_duration"]].append(test2[["user_id","media_id","ts_listen","provenencia","media_duration"]], ignore_index=True)
temp["ts_listen2"]=pd.to_datetime(temp["ts_listen"], unit='s')
del train2
del test2
temp=temp.sort_values(by=["user_id","ts_listen2"],ascending=[True,True])
temp=temp.reset_index(drop=True)


In [25]:
temp.loc[:,"tiempo_anterior"]=temp.loc[:,"ts_listen2"].diff()/ np.timedelta64(1, 's')
temp["dif_user"]=temp["user_id"].diff()
temp.loc[temp["dif_user"]!=0.0,"tiempo_anterior"]=np.nan
temp["comparacion"]=temp["media_duration"].shift(1) # se comparara con la duracion de la cancion anterior
# state recomendation, recibira el valor de 1 si es flow
temp.loc[:,"state_recomendation"]=0
temp.loc[((temp["tiempo_anterior"]-temp["comparacion"])>15) | (temp["tiempo_anterior"].isnull()),"state_recomendation"]=1

train=train.merge(temp[["user_id","ts_listen","state_recomendation"]],on=["user_id","ts_listen"],how="left")
test=test.merge(temp[["user_id","ts_listen","state_recomendation"]],on=["user_id","ts_listen"],how="left")

del temp

In [26]:
train.loc[pd.to_datetime(train["ts_listen"], unit='s').dt.year!=2016,["ts_listen","state_recomendation"]]=np.nan
test.loc[pd.to_datetime(test["ts_listen"], unit='s').dt.year!=2016,"ts_listen"]=np.nan



temp = train[["user_id","media_id","ts_listen","listen_type"]].append(test[["user_id","media_id","ts_listen","listen_type"]], ignore_index=True)
temp["ts_listen"]=pd.to_datetime(temp["ts_listen"], unit='s')

temp=temp.groupby(["user_id","media_id","listen_type"]).agg(["min"]).reset_index()

temp.columns=temp.columns.droplevel(1)
#temp.columns.values[3] ="date_min_song"

temp_recomendado=temp.loc[temp.loc[:,"listen_type"]==1,]
temp_buscado=temp.loc[temp.loc[:,"listen_type"]==0,]

temp_recomendado.rename(columns={'ts_listen': 'date_min_song_recom'}, inplace=True)
temp_buscado.rename(columns={'ts_listen': 'date_min_song_busq'}, inplace=True)

temp_recomendado=temp_recomendado.reset_index(drop=True)
temp_buscado=temp_buscado.reset_index(drop=True)

del temp_recomendado["listen_type"]
del temp_buscado["listen_type"]

train=train.merge(temp_recomendado,on=["user_id","media_id"],how="left")
train=train.merge(temp_buscado,on=["user_id","media_id"],how="left")

test=test.merge(temp_recomendado,on=["user_id","media_id"],how="left")
test=test.merge(temp_buscado,on=["user_id","media_id"],how="left")

train["ya_buscado"]=1
train.loc[(train["date_min_song_busq"].isnull()) | (pd.to_datetime(train["ts_listen"], unit='s')==train["date_min_song_busq"]),"ya_buscado"]=0 

train["ya_buscado"]=train['ya_buscado'].map({True: 1, False: 0 })

#train["ya_buscado"]*1

test["ya_buscado"]=1
test.loc[(test["date_min_song_busq"].isnull()) | (pd.to_datetime(test["ts_listen"], unit='s')==test["date_min_song_busq"]),"ya_buscado"]=0 

#test["ya_buscado]=(test["date_min_song_busq"].isnull()) | (pd.to_datetime(test["ts_listen"], unit='s')==test["date_min_song_busq"])*1

test["ya_buscado"]=test['ya_buscado'].map({True: 1, False: 0 })


del temp_recomendado
del temp_buscado

C:\Program Files\Anaconda2\lib\site-packages\pandas\core\frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [27]:
train.columns

Index([u'genre_id', u'ts_listen', u'media_id', u'album_id', u'context_type',
       u'release_date', u'platform_name', u'platform_family',
       u'media_duration', u'listen_type', u'user_gender', u'user_id',
       u'artist_id', u'user_age', u'is_listened', u'state_recomendation',
       u'date_min_song_recom', u'date_min_song_busq', u'ya_buscado'],
      dtype='object')

In [28]:
# checkear rolling mean pandas, y extraer mas datos de las busquedas que el usuario hace

noflow=train.loc[train.loc[:,"listen_type"]!=1,]

noflow["ts_listen"]=pd.to_datetime(train["ts_listen"], unit='s')


f = {"ts_listen":["size"],"is_listened":["sum","mean"]}

#pd.to_datetime(train["ts_listen"], unit='s')
### 
freq_canc=noflow[["ts_listen","is_listened","user_id","media_id"]].groupby(["user_id","media_id"]).agg(f).reset_index()
freq_canc.columns=freq_canc.columns.droplevel(1)
freq_canc.columns.values[2] ="freq_cancion_buscada"
freq_canc.columns.values[3] ="freq_cancion_buscada_escuchada"
freq_canc.columns.values[4] ="ratio_cancion_buscada_escuchada"



mean_music_user=noflow[["user_id","media_duration"]].groupby("user_id").mean().reset_index()
mean_music_user.columns.values[1]="mean_user_busc"

mean_age_song=noflow[["media_id","user_age"]].groupby("media_id").mean().reset_index()
mean_age_song.columns.values[1]="song_age_busc"

### tambien agregaremos el número de busquedas que tiene una canción
num_buc=pd.DataFrame(noflow[["media_id"]].groupby(["media_id"]).size().reset_index())
num_buc.columns.values[1]="num_busquedas"

## tambien agregaremos el numero de busquedas por genero para cada usuario y el numero de generos que busca
num_gen_busc=pd.DataFrame(noflow[["user_id","genre_id","is_listened"]].groupby(["user_id","genre_id"]).agg(["size","mean"]).reset_index())
num_gen_busc.columns = num_gen_busc.columns.get_level_values(0)
num_gen_busc.columns.values[2]= "num_busquedas_porgenero"
num_gen_busc.columns.values[3]="ratio_busqueda_porgenero_escuchado"


num_gen_div=noflow[["user_id","genre_id"]].groupby(["user_id"]).agg(["nunique"]).reset_index()
num_gen_div.columns = num_gen_div.columns.get_level_values(0)
num_gen_div.columns.values[1]="num_generos_buscados"




del noflow

#s.columns=s.columns.droplevel(1)




C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
### probablemente hay canciones que casi nadie escucha o que es seguro que la escuchen todos, seria buena crear una variable que recoja eso
### tambien ranking de canciones mas buscadas
# explorar el album

### aqui tambien debemos agregar si ya la busco , en caso no la haya buscado na
### tambien agregar de las busquedas, cuantas llegaron a ser escuchadas,tanto a nivel usuario, como total o por cancion


# to do, relacionr hace cuanyos dias la escucho por usuario

train=train[train["listen_type"]==1]


train=train.merge(freq_canc,on=["user_id","media_id"],how="left")
train["freq_cancion_buscada"]=train["freq_cancion_buscada"].fillna(0)
train["freq_cancion_buscada_escuchada"]=train["freq_cancion_buscada_escuchada"].fillna(0)
train["ratio_cancion_buscada_escuchada"]=train["ratio_cancion_buscada_escuchada"].fillna(0)





test=test.merge(freq_canc,on=["user_id","media_id"],how="left")
test["freq_cancion_buscada"]=test["freq_cancion_buscada"].fillna(0)
test["freq_cancion_buscada"]=train["freq_cancion_buscada"].fillna(0)
test["freq_cancion_buscada_escuchada"]=test["freq_cancion_buscada_escuchada"].fillna(0)
test["ratio_cancion_buscada_escuchada"]=test["ratio_cancion_buscada_escuchada"].fillna(0)

train=train.merge(mean_music_user,on="user_id",how="left")
test=test.merge(mean_music_user,on="user_id",how="left")

train=train.merge(mean_age_song,on="media_id",how="left")
test=test.merge(mean_age_song,on="media_id",how="left")

train=train.merge(num_buc,on="media_id",how="left")
test=test.merge(num_buc,on="media_id",how="left")
train[["num_busquedas"]].fillna(0,inplace=True)
test[["num_busquedas"]].fillna(0,inplace=True)




train=train.merge(num_gen_busc,on=["user_id","genre_id"],how="left")
test=test.merge(num_gen_busc,on=["user_id","genre_id"],how="left")
train[["num_busquedas_porgenero","ratio_busqueda_porgenero_escuchado"]].fillna(0,inplace=True)
test[["num_busquedas_porgenero","ratio_busqueda_porgenero_escuchado"]].fillna(0,inplace=True)
                                                                        
train=train.merge(num_gen_div,on=["user_id"],how="left")
test=test.merge(num_gen_div,on=["user_id"],how="left")
train[["num_generos_buscados"]].fillna(0,inplace=True)
test[["num_generos_buscados"]].fillna(0,inplace=True)






train["tiempo_recomendado"]=(pd.to_datetime(train["ts_listen"], unit='s')-train["date_min_song_recom"]).dt.days
test["tiempo_recomendado"]=(pd.to_datetime(test["ts_listen"], unit='s')-test["date_min_song_recom"]).dt.days

train["ya_recomendado"]= pd.to_datetime(train["ts_listen"], unit='s')!=train["date_min_song_recom"]
test["ya_recomendado"]= pd.to_datetime(test["ts_listen"], unit='s')!=test["date_min_song_recom"]

train["ya_recomendado"]=train['ya_recomendado'].map({True: 1, False: 0 })
test["ya_recomendado"]=test['ya_recomendado'].map({True: 1, False: 0 })

train["ya_interactuo"]=0
train.loc[(train["ya_recomendado"]==1) | (train["ya_buscado"]==1),"ya_interactuo"]=1

test["ya_interactuo"]=0
test.loc[(test["ya_recomendado"]==1) | (test["ya_buscado"]==1),"ya_interactuo"]=1

train=train.merge(json_music[["media_id","Remastered_song","Tribute to_song","Version_song","Edition_song","Deluxe_song",\
"Special_song","Remix_song","Feat_song","Live_song","letters_count_song","words_count_song","letters_count/words_count",\
                   "quantile_letters_count_song","quantile_words_count_song","quantile_letters_count/words_count"]],on=["media_id"],how="left")

test=test.merge(json_music[["media_id","Remastered_song","Tribute to_song","Version_song","Edition_song","Deluxe_song",\
"Special_song","Remix_song","Feat_song","Live_song","letters_count_song","words_count_song","letters_count/words_count",\
                         "quantile_letters_count_song","quantile_words_count_song","quantile_letters_count/words_count"  ]],on=["media_id"],how="left")




### crear variables que recojan info de los 2 ultimos años para luego cruzarlas con el usuario
### 2016,20015,2013y20014,20012y 2011, 2010 para abajo # luego arreglar 2017

for df in train,test:
    df["hour_listened"]=pd.to_datetime(df["ts_listen"], unit='s').dt.hour
    df["minute_listened"]=pd.to_datetime(df["ts_listen"], unit='s').dt.minute
    df["weekday_listened"]=pd.to_datetime(df["ts_listen"], unit='s').dt.dayofweek
    
    df["year_released"]=pd.to_datetime(df["release_date"], format="%Y%m%d",errors="coerce").dt.year
    df["year_category"]=pd.cut(df["year_released"],bins=[-1,2009,2011,2013,2014,2016])
    df["decade_released"]=(df["year_released"]//10)*10
    df.loc[df["decade_released"]<1960.0,"decade_released"]="Otros"
    df["month_released"]=pd.to_datetime(df["release_date"], format="%Y%m%d",errors="coerce").dt.month
    
    df['daytime_listened'] = df["hour_listened"].apply(classify_daytime)
    
    df["ratio_age_songuser"]=(2016-df["year_released"])*1.0/(df["user_age"])
    
    
def red2(data, variable,pcategorias=0.01):
        u=set(data[variable].dropna())
        l=[]
        #l reunira a todos los con len mayor al trheshold
        for i in u:
            if len(data[data[variable]==i]) > len(data)*(pcategorias) :
                l.append(i)
        
        
        
        data.loc[((~data[variable].isin(l))&(data[variable].notnull())),variable]="Otros"
        
        return "Variable "+variable+" convertida"

### el cambio en el genre id tiene que ser pra toda la data
temp=train[["genre_id"]].append(test[["genre_id"]], ignore_index=True)
    
red2(temp,"genre_id")

train["genre_id"]=temp.loc[0:len(train),"genre_id"]
temp=temp.loc[len(train):]
temp.reset_index(drop=True,inplace=True)
test["genre_id"]=temp["genre_id"]

# context type muestra desigualdad entre el train y test, por el momento no sera considerado
#temp=train[["context_type"]].append(test[["context_type"]], ignore_index=True)
#red2(temp,"context_type",pcategorias=0.0001)
#train["context_type"]=temp.loc[0:len(train),"context_type"]
#temp=temp.loc[len(train):]
#temp.reset_index(drop=True,inplace=True)
#test["context_type"]=temp["context_type"]

del temp


train["ts_listen2"]=pd.to_datetime(train["ts_listen"], unit='s')
test["ts_listen2"]=pd.to_datetime(test["ts_listen"], unit='s')

#primero daremos los orden de canciones para cada usuario
temp=train[["user_id","ts_listen","ts_listen2"]].append(test[["user_id","ts_listen","ts_listen2"]],ignore_index=True)
temp=temp.loc[temp.loc[:,"ts_listen2"].dt.year==2016,]


temp.sort_values(["user_id","ts_listen2"],ascending=[True,True],inplace=True)
temp.reset_index(drop=True,inplace=True)
temp["indicator"]=1
temp["order_user_song"]=temp.groupby("user_id")["indicator"].cumsum()

temp.sort_values(["user_id","ts_listen2"],inplace=True)
temp.reset_index(drop=True,inplace=True)
temp["indicator"]=1
temp["order_user_song"]=temp.groupby("user_id")["indicator"].cumsum()

temp=temp.sort_values("ts_listen2",ascending=True)
temp.reset_index(drop=True,inplace=True)
temp["general_order"]=temp["indicator"].cumsum()

train=train.merge(temp[["user_id","ts_listen","order_user_song","general_order"]],on=["user_id","ts_listen"],how="left")
test=test.merge(temp[["user_id","ts_listen","order_user_song","general_order"]],on=["user_id","ts_listen"],how="left")





#train_train, train_test = train_test_split(train, test_size = 0.1,random_state=123)
train=train.sort_values("ts_listen2",ascending=True)
train.reset_index(drop=True,inplace=True)
#train_train=train.loc[0:len(train)*9/10]
#train_test=train.loc[len(train)*9/10 : ]
#train_test.reset_index(drop=True,inplace=True)



train,test=estimate_cross(train=train,test=test,var_cat=["media_id"],target=["is_listened"])

#train_train,train_test,test=cat_target(var_cat=["media_id"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["album_id"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["album_id"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)


#train_train,train_test,test=cat_target(var_cat=["album_id","state_recomendation"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["album_id","state_recomendation"],target=["is_listened"])



#train_train,train_test,test=cat_target(var_cat=["album_id","ya_recomendado"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["album_id","ya_recomendado"],target=["is_listened"])

#train_train,train_test,test=cat_target(var_cat=["user_id"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id"],target=["is_listened"])

#train_train,train_test,test=cat_target(var_cat=["artist_id"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["artist_id"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","year_category"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","year_category"],target=["is_listened"])

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","quantile_letters_count_song"],target=["is_listened"])

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","quantile_words_count_song"],target=["is_listened"])

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","quantile_letters_count/words_count"],target=["is_listened"])


#"quantile_letters_count_song","quantile_words_count_song","quantile_letters_count/words_count"

#train_train,train_test,test=cat_target(var_cat=["genre_id","year_category"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["genre_id","year_category"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","decade_released"],target=["is_listened"],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","decade_released"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","hour_listened"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","hour_listened"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id",'daytime_listened'],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id",'daytime_listened'],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id",'state_recomendation'],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id",'state_recomendation'],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","hour_listened","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","hour_listened","state_recomendation"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","daytime_listened","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","daytime_listened","state_recomendation"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","weekday_listened"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","weekday_listened"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","weekday_listened","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","weekday_listened","state_recomendation"],target=["is_listened"])

#train_train,train_test,test=cat_target(var_cat=["user_id","artist_id"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","artist_id"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","media_id"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

#train,test=estimate_cross(train=train,test=test,var_cat=["user_id","media_id"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["media_id","hour_listened"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["media_id","hour_listened"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["media_id",'daytime_listened'],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["media_id",'daytime_listened'],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["media_id",'daytime_listened',"state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["media_id",'daytime_listened',"state_recomendation"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","genre_id"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","genre_id"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["genre_id","daytime_listened"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["genre_id","daytime_listened"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","platform_family"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","platform_family"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","platform_family","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","platform_family","state_recomendation"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","platform_family","ya_recomendado"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","platform_family","ya_recomendado"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","platform_family","ya_buscado"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","platform_family","ya_buscado"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","platform_name"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","platform_name"],target=["is_listened"])

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","platform_family","platform_name"],target=["is_listened"])



#train_train,train_test,test=cat_target(var_cat=["user_id","decade_released"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)



#train_train,train_test,test=cat_target(var_cat=["user_id","tiempo_recomendado"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","tiempo_recomendado"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","ya_recomendado"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","ya_recomendado"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","ya_buscado"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","ya_buscado"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","ya_interactuo"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","ya_interactuo"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["hour_listened","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

#train_train,train_test,test=cat_target(var_cat=["media_id","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["media_id","state_recomendation"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","ya_recomendado","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","ya_recomendado","state_recomendation"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","ya_buscado","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","ya_buscado","state_recomendation"],target=["is_listened"])


#train_train,train_test,test=cat_target(var_cat=["user_id","ya_interactuo","state_recomendation"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","ya_interactuo","state_recomendation"],target=["is_listened"])



#train_train,train_test,test=cat_target(var_cat=["user_id","context_type"],target=['is_listened'],param_train=train_train,param_test=train_test,test_ofic=test)

train,test=estimate_cross(train=train,test=test,var_cat=["user_id","context_type"],target=["is_listened"])


# aqui recien se hace la particion

#train_train=train.loc[0:len(train)*9/10]
#train_test=train.loc[len(train)*9/10 : ]
#train_test.reset_index(drop=True,inplace=True)


# recordar que estas funciones no solo las podemos cruzar con el is_listened

#train_train,train_test,test=compare_target(unidades=["user_id"],variable=["media_duration"],param_train=train_train,param_test=train_test,test_ofic=test,target=["is_listened"],function="mean")

#train_train,train_test,test=compare_target(unidades=["user_id"],variable=["year_released"],param_train=train_train,param_test=train_test,test_ofic=test,target=["is_listened"],function="mean")

#train_train,train_test,test=compare_target(unidades=["media_id"],variable=["user_age"],param_train=train_train,param_test=train_test,test_ofic=test,target=["is_listened"],function="mean")



# probar con context-type

C:\Program Files\Anaconda2\lib\site-packages\pandas\core\frame.py:2842: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [1]:
import pandas as pd

In [30]:
#train[["sng_title","words_count_song"]]

#json_music

pd.crosstab(pd.cut(train["words_count_song"],4),train["is_listened"])


is_listened,0,1
words_count_song,,
"(-0.03, 7.5]",907755,1363896
"(7.5, 15]",18969,27827
"(15, 22.5]",449,620
"(22.5, 30]",18,77


In [31]:
train[["Remastered_song","Tribute to_song","Version_song","Edition_song","Deluxe_song",\
"Special_song","Remix_song","Feat_song","Live_song"]].sum(axis=1).quantile([0.75,0.8,0.85,0.9,0.95,0.99,1])

0.75    0.0
0.80    0.0
0.85    0.0
0.90    0.0
0.95    0.0
0.99    1.0
1.00    2.0
dtype: float64

In [32]:
train["letters_count/words_count"].describe()

count    2.319607e+06
mean     5.379513e+00
std      1.777488e+00
min      1.000000e+00
25%      4.125000e+00
50%      5.000000e+00
75%      6.000000e+00
max      3.600000e+01
Name: letters_count/words_count, dtype: float64

In [33]:
pd.crosstab(pd.qcut(train["letters_count/words_count"],5),train["is_listened"])

is_listened,0,1
letters_count/words_count,,
"[1, 4]",236411,342976
"(4, 4.667]",155019,234534
"(4.667, 5.333]",175891,261611
"(5.333, 6.5]",192627,289768
"(6.5, 36]",167241,263529


In [36]:
# creando variables de resta de fechas

#for df in train,test:

#    df["transc_busqueda"]=(pd.to_datetime(df["fecha_maxima"], unit='s')-pd.to_datetime(df["fecha_minima"], unit='s')).dt.days
#    df["trans_busqmax_rec"]=(pd.to_datetime(df["fecha_maxima"], unit='s')-pd.to_datetime(df["ts_listen"], unit='s')).dt.days
#    df["trans_rec_busqmin"]=(pd.to_datetime(df["ts_listen"], unit='s')-pd.to_datetime(df["fecha_minima"], unit='s')).dt.days

### 2016,20015,2013y20014,20012y 2011, 2010 para abajo # luego arreglar 2017

for df in train,test:

    df["ratio_musictime_busc"]=df["media_duration"]*1.0/df["mean_user_busc"]
    df["ratio_age_busc"]=df["user_age"]*1.0/df["song_age_busc"]
    #df["ratio_musictime_listen"]=df["media_duration"]*1.0/df["media_duration_target"]
    #df["ratio_age_listen"]=df["user_age"]*1.0/df["user_age_target"]
    #df["dif_year_pref"]=df["year_released"]-df["year_released_target"]
    df["timeratio_user_artist_prob"]=df['user_id-hour_listened_prob_target']/df['media_id-hour_listened_prob_target']
    
    ### estadisticas de las variables expresada como probabilidades
    prob_variables = [col for col in train.columns if 'prob_target' in col]
    prob_user_variables = [col for col in train.columns if ('prob_target' in col) and ("user_id" in col)]

    
    df['mean_prob'] = df[prob_variables].mean(axis=1,skipna=True)
    df['std_prob'] = df[prob_variables].std(axis=1,skipna=True)
    df["num_na_prob"]=df[prob_variables].isnull().sum(axis=1)
    
    df["mean_user_prob"]=df[prob_user_variables].mean(axis=1,skipna=True)
    df['std_user_prob'] = df[prob_user_variables].std(axis=1,skipna=True)
    df["num_na_user_prob"]=df[prob_user_variables].isnull().sum(axis=1)
    df["ratio_prob_user"]=df["mean_user_prob"]/df["mean_prob"]


    #df["user_inter"]=df[['user_id-ya_recomendado_prob_target','user_id-ya_buscado-state_recomendation_prob_target','user_id-ya_interactuo-state_recomendation_prob_target']].mean(axis=1)
    df["user_inter"]=df[['user_id-ya_recomendado-state_recomendation_prob_target','user_id-ya_buscado-state_recomendation_prob_target','user_id-ya_interactuo-state_recomendation_prob_target']].mean(axis=1)
    
    df["ratio_pro_uer"]=df["mean_user_prob"]/df["mean_prob"]
    df["ratio_media_user"]=(df["media_id_prob_target"])/(df['user_id_prob_target'])
    df["ratio_artist_user"]=(df["artist_id_prob_target"])/(df['user_id_prob_target'])
    df["ratio_artist_hour"]=(df['user_id-artist_id_prob_target'])/(df['user_id-hour_listened_prob_target'])
    df["timeratio_user_artist_prob"]=(df['user_id-hour_listened_prob_target'])/(df['media_id-hour_listened_prob_target'])
    df["ratio_yarec_state"]=(df['user_id-ya_recomendado_prob_target'])/(df['user_id-state_recomendation_prob_target'])
    df["ratio_context_user"]=(df['user_id-context_type_prob_target'])/(df['user_id_prob_target'])
    #df["ratio_album_state"]=(df['album_id-state_recomendation_prob_target'])/(df['album_id_prob_target'])
    #df["ratio_album_ya"]=(df['album_id-ya_recomendado_prob_target'])/(df['album_id_prob_target'])
    df["ratio_user_ya"]=(df['user_id_prob_target'])/(df['user_id-ya_recomendado_prob_target'])
    df["ratio_user_state"]=(df['user_id_prob_target'])/(df['user_id-state_recomendation_prob_target'])
    #df["ratio_hour_media_state"]=(df['hour_listened-state_recomendation_prob_target']+1)/(df['media_id-state_recomendation_prob_target']+1)
    


 


    





In [38]:
# depues ver context typr
## examinar bien genre_id
### explorar mejor los años
#varcat=["genre_id","platform_name","platform_family","decade_released",'daytime_listened']
varcat=["platform_name","platform_family",'daytime_listened']

#varnum=["media_id","media_duration","listen_type","user_gender","user_age","hour_listened","year_released","weekday_listened","month_released"]
#varnum=["user_id","media_id","media_duration","context_type","user_gender","user_age","hour_listened",'minute_listened',"year_released",\
#        'month_released',"freq_cancion_buscada",'freq_cancion_buscada_escuchada','ratio_cancion_buscada_escuchada',\
#        "mean_user_busc","song_age_busc","ratio_musictime_busc","ratio_age_busc",\
#       "media_duration_target","user_age_target","year_released_target","ratio_musictime_listen","ratio_age_listen","dif_year_pref",\
#       "num_busquedas","ratio_age_songuser",'media_id_prob_target','media_id-hour_listened_prob_target',\
#       'tiempo_recomendado','user_id-genre_id_prob_target',\
#       "ya_recomendado","ya_buscado","ya_interactuo",'genre_id-platform_family_prob_target','user_id-decade_released_prob_target',\
#        'user_id-tiempo_recomendado_prob_target',"user_id-ya_recomendado_prob_target",'user_id-platform_name_prob_target',\
#   'user_id-hour_listened_prob_target','user_id-platform_family_prob_target',"timeratio_user_artist_prob",\
#    'user_id-state_recomendation_prob_target','hour_listened-state_recomendation_prob_target','user_id-context_type_prob_target',\
#    'media_id-state_recomendation_prob_target',"user_id-ya_recomendado-state_recomendation_prob_target",\
#      'user_id-ya_buscado-state_recomendation_prob_target','user_id-ya_interactuo-state_recomendation_prob_target', \
#        'hour_listened-ya_buscado_prob_target','hour_listened-ya_interactuo_prob_target',"state_recomendation"]

novarnum=varcat+["genre_id","decade_released","album_id","ts_listen","release_date","listen_type","artist_id","is_listened",'ts_listen2',"date_min_song_recom","date_min_song_busq","year_category",\
                "Remastered_song","Tribute to_song","Version_song","Edition_song","Deluxe_song",\
                "quantile_letters_count_song","quantile_words_count_song","quantile_letters_count/words_count"]# por ahora no consideraremos las categorias de las ultimas 2 lineas
varnum=[x for x in train.columns if x not in novarnum]

varcomp=varcat+varnum

target=["is_listened"]





In [41]:
# skill de una cancion por hora o incluso mas condiciones
## tambien se puede agregar numero de busquedas de cada cancion en cada dispositivo, a cuantos usuarios diferentes se recomienda
## la cancion, o cuantos buscan la cancion
# skill de la hora, probabilidad de q cualquier cancion se escuchada por hora, cuantos distintos usuarios aceptan las canciones, etc
# remastered
# 39 iteration 0.7275
# al poner mean prob, que trabaje con skipna=False, dejo de ser la variable mas importante y se fue para abajo, y el auc en el leaderboard
# bajo, comparando submission 51 y 49
x=np.array(pd.get_dummies(train.loc[:,varcomp],columns=varcat))
y=np.array(train.loc[:,target]).ravel()



tx_start=np.array(pd.get_dummies(train.loc[train["state_recomendation"]==1,varcomp],columns=varcat))
ty_start=np.array(train.loc[train["state_recomendation"]==1,target]).ravel()

## con este salian resultados interesantes 
# XGBClassifier(n_estimators=2500, objective= 'binary:logistic', seed=27,learning_rate= 0.015, colsample_bytree=0.8, max_depth= 12, subsample=0.9,min_child_weight=100)

xg=XGBClassifier(n_estimators=2500, objective= 'binary:logistic', seed=27,learning_rate= 0.015, colsample_bytree=0.6, max_depth= 10, subsample=0.9)

xg.fit(x,y,eval_set=[(tx_start,ty_start),(x,y)], eval_metric="auc", verbose=True,early_stopping_rounds=100)


[0]	validation_0-auc:0.701344	validation_1-auc:0.8305
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.711798	validation_1-auc:0.835493
[2]	validation_0-auc:0.714304	validation_1-auc:0.83682
[3]	validation_0-auc:0.715137	validation_1-auc:0.836683
[4]	validation_0-auc:0.715741	validation_1-auc:0.836535
[5]	validation_0-auc:0.717671	validation_1-auc:0.836825
[6]	validation_0-auc:0.718769	validation_1-auc:0.837097
[7]	validation_0-auc:0.719072	validation_1-auc:0.838223
[8]	validation_0-auc:0.719287	validation_1-auc:0.838853
[9]	validation_0-auc:0.719289	validation_1-auc:0.838697
[10]	validation_0-auc:0.719869	validation_1-auc:0.839069
[11]	validation_0-auc:0.720217	validation_1-auc:0.839397
[12]	validation_0-auc:0.720106	validation_1-auc:0.839687
[13]	validation_0-auc:0.719931	validation_1-auc:0.839609
[14]	validation_0-auc:0.720272	validation_1-auc:0.839549


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0, learning_rate=0.015, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=2500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.9)

In [45]:
## si el modelo para, usar preds = model.predict(xgtest, ntree_limit=bst.best_ntree_limit)


def impxg(dict,variables):
    dictp={ int(a.replace("f","")): dict[a] for a in dict.keys() }
    dictimp={variables[a]:dictp[a] for a in dictp.keys()}
    xgimp=sorted(dictimp.items(), key=lambda x: x[1],reverse=True)

    return xgimp

ixg=impxg(xg.booster().get_score(fmap='', importance_type="gain"),pd.get_dummies(train[varcomp],columns=varcat).columns)

ixg

[('mean_prob', 559.4566224810206),
 ('mean_user_prob', 397.6400166663245),
 ('user_id_prob_target', 248.11401808258506),
 ('user_id-ya_buscado-state_recomendation_prob_target', 144.37618809239652),
 ('platform_family_0', 133.5896979478765),
 ('user_id-platform_family-platform_name_prob_target', 110.98365089768859),
 ('platform_family_1', 98.33066649507168),
 ('user_id-platform_family-ya_buscado_prob_target', 92.61750820740032),
 ('ratio_cancion_buscada_escuchada', 90.38560480484045),
 ('platform_name_2', 86.7264040715861),
 ('state_recomendation', 79.18615467366934),
 ('user_id-platform_family-state_recomendation_prob_target',
  71.65208956791538),
 ('user_id-quantile_letters_count/words_count_prob_target', 71.42351238534324),
 ('freq_cancion_buscada_escuchada', 58.107619861776456),
 ('user_id-quantile_words_count_song_prob_target', 48.938118325131086),
 ('ratio_busqueda_porgenero_escuchado', 43.9752303096837),
 ('ya_recomendado', 43.03293189327437),
 ('user_id-platform_name_prob_targe

In [ ]:
# to do: ratio entre edad y antiguedad de la cancion, variable edad promedio del que escucha la cancion y rango de edades, 
#si esta escuchando en su plataforma tipica
# decada de la cancion


In [42]:
# submission 55, detecto un error al ordenar la data para mandar a train, asi que lo volvimo a entrar corrigiendolo para mandar submisiion 56
# lo mismo con submission 56, al parecer tiempo anterior trae overfitting
# sacanadole dicha variable, en submission 57, el overfitting ya no habia
tsx=np.array(pd.get_dummies(test[varcomp],columns=varcat))
kx=xg.predict_proba(tsx,ntree_limit=xg.best_ntree_limit)
test["is_listened"]=kx[:,1]

In [43]:
test[["sample_id","is_listened"]].to_csv("submission109.csv",index=False)